In [2]:
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

In [108]:
Amazon = pd.read_csv('Datasets/amazon_prime_titles.csv') #9668 registros
Disney = pd.read_csv('Datasets/disney_plus_titles.csv') #1450 registros
Hulu = pd.read_csv('Datasets/hulu_titles.csv') #3073 registros
Netflix = pd.read_json('Datasets/netflix_titles.json') #8807 registros

In [109]:
#insertamos columna con la plataforma correspondiente
Amazon['Platform'] = 'Amazon'
Disney['Platform'] = 'Disney'
Hulu['Platform'] = 'Hulu'
Netflix['Platform'] = 'Netflix'

In [110]:
#Construimos dataset con todas las plataformas
Total_Data = pd.concat([Amazon, Disney, Hulu, Netflix])
Total_Data.reset_index(drop=True, inplace=True)

In [111]:
#Eliminamos columnas que no necesitaremos para la consigna requerida
TD_1 = Total_Data.drop(['show_id', 'country', 'director', 'date_added', 'rating', 'description'], axis=1)

In [112]:
#convertimos el tipo de dato de la columna 'duration' a datos numericos
TD_1['duration'] = TD_1['duration'].str.replace('min' , '')
TD_1['duration'] = TD_1['duration'].str.replace('Seasons' , '')
TD_1['duration'] = TD_1['duration'].str.replace('Season' , '')
TD_1['duration'] = pd.to_numeric(TD_1['duration'])


In [114]:
#Consulta de prueba exitosa
Filtro = (TD_1['Platform'] == 'Hulu') & (TD_1['release_year'] == 2018) & (TD_1['type'] == 'Movie')
elemento = TD_1[Filtro]['duration'].idxmax()
TD_1.loc[elemento,'title']

'The House That Jack Built'

In [106]:
#Función conteo de Series y Peliculas
condicion = (TD_1['Platform']=='Netflix')
#TD_2 = TD_1[condicion]
Nro_Movies = TD_1[condicion]['type'].str.contains(pat = 'Movie').sum()
Nro_TVShows = TD_1[condicion]['type'].str.contains(pat ='TV Show').sum()
print(Nro_Movies,Nro_TVShows)

6131 2676


In [68]:
#Consulta de prueba 3
Filtro = TD_1['listed_in'].str.contains('Comedy')
Nro_Comedias = TD_1[Filtro].groupby('Platform')['listed_in'].count()
print(Nro_Comedias)

Platform
Amazon     2099
Disney      526
Hulu        667
Netflix     399
Name: listed_in, dtype: int64


In [125]:
#Consulta Nro 4

condicion = (TD_1['Platform']=='Netflix') & (TD_1['release_year']==2018) 
actores = TD_1[condicion]['cast'].str.split(pat =',', expand=True)
union = pd.concat(objs = [actores[i] for i in range(len(actores.columns))], ignore_index = True)
union.value_counts()




 Andrea Libman        8
 Rupa Bhimani         6
 Vincent Tong         6
 Jigna Bharadhwaj     6
 Takahiro Sakurai     6
                     ..
 T.J. Miller          1
 Beau Knapp           1
 Géraldine Pailhas    1
 Richard Alan Reid    1
 Tom Holland          1
Length: 6825, dtype: int64

In [214]:
pd.Series(TD_2['cast'].values.flatten()).str.contains('Derren Brown').sum()

3

In [215]:
pd.Series(TD_2['cast'].values.flatten()).str.contains('Andrea Libman').sum()

8

In [219]:
#Normalizamos columna 'duration' a través de filtros
#Duration_TVShow =(TD_1['type']=='TV Show') 
#TD_1['N°Sesions'] = TD_1['duration'][Duration_TVShow]
#Duration_Movie = (TD_1['type']== 'Movie')
#TD_1['Movie_Duration(min)']=TD_1['duration'][Duration_Movie]
#DF_Normalizado=TD_1.drop('duration', axis=1)

In [221]:
#Funciones
def MaxDuracion(año, tipo, plataforma):
#years = list(DF_Normalizado['release_year'].unique())
#platforms = ['Amazon','Disney','Hulu', 'Netflix']
    Filtro = (TD_1['Platform'] == plataforma) & (TD_1['release_year'] == año) & (TD_1['type'] == tipo)
    indice = TD_1[Filtro]['duration'].idxmax()
    return TD_1.loc[indice,'duration']


In [45]:
TD_1

,type,title,cast,release_year,duration,listed_in,Platform
0,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113.0,"Comedy, Drama",Amazon
1,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110.0,"Drama, International",Amazon
2,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74.0,"Action, Drama, Suspense",Amazon
3,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69.0,Documentary,Amazon
4,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45.0,"Drama, Fantasy",Amazon
...,...,...,...,...,...,...,...
22993,Movie,Zodiac,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",2007,158.0,"Cult Movies, Dramas, Thrillers",Netflix
22994,TV Show,Zombie Dumb,None,2018,2.0,"Kids' TV, Korean TV Shows, TV Comedies",Netflix
22995,Movie,Zombieland,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",2009,88.0,"Comedies, Horror Movies",Netflix
22996,Movie,Zoom,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",2006,88.0,"Children & Family Movies, Comedies",Netflix


In [101]:
#Generamos Base de Datos

engine =sqlalchemy.create_engine('sqlite:///Datasets/filmes.db')
TD_1.to_sql(name='filmes', con = engine) 


22998

In [102]:
pd.read_sql('filmes', engine)

,index,type,title,cast,release_year,duration,listed_in,Platform
0,0,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113.0,"Comedy, Drama",Amazon
1,1,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110.0,"Drama, International",Amazon
2,2,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74.0,"Action, Drama, Suspense",Amazon
3,3,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69.0,Documentary,Amazon
4,4,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45.0,"Drama, Fantasy",Amazon
...,...,...,...,...,...,...,...,...
22993,22993,Movie,Zodiac,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",2007,158.0,"Cult Movies, Dramas, Thrillers",Netflix
22994,22994,TV Show,Zombie Dumb,None,2018,2.0,"Kids' TV, Korean TV Shows, TV Comedies",Netflix
22995,22995,Movie,Zombieland,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",2009,88.0,"Comedies, Horror Movies",Netflix
22996,22996,Movie,Zoom,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",2006,88.0,"Children & Family Movies, Comedies",Netflix


In [58]:
create_sql = "CREATE TABLE IF NOT EXISTS filmes (type TEXT, title TEXT, cast TEXT, release_year INTEGER, duration INTEGER, listed in TEXT, Platform TEXT)"
cursor=conn.cursor()
cursor.execute(create_sql)

OperationalError: near "in": syntax error

In [56]:
cursor.execute(create_sql)

OperationalError: near "in": syntax error

In [102]:
condicion = (TD_1['type'] == 'TV Show')
TD_1[condicion]

,type,title,cast,release_year,duration,listed_in,Plataforma
17,TV Show,Zoo Babies,Narrator - Gillian Barlett,2008,1,"Kids, Special Interest",Amazon
18,TV Show,Zoë Coombs Marr: Bossy Bottom,Zoë Coombs Marr,2020,1,"Comedy, Talk Show and Variety",Amazon
20,TV Show,Zoboomafoo,NaN,2001,1,Kids,Amazon
21,TV Show,Zo Zo Zombie: Mini-Series,"Alex Cazares, Ben Diskin, Cristina Vee, Erika ...",2020,1,"Action, Anime, Comedy",Amazon
23,TV Show,ZeroZeroZero,"ANDREA RISEBOROUGH, HAROLD TORRES, CLAUDIA PIN...",2020,1,TV Shows,Amazon
...,...,...,...,...,...,...,...
8795,TV Show,Yu-Gi-Oh! Arc-V,"Mike Liscio, Emily Bauer, Billy Bob Thompson, ...",2015,2,"Anime Series, Kids' TV",Netflix
8796,TV Show,Yunus Emre,"Gökhan Atalay, Payidar Tüfekçioglu, Baran Akbu...",2016,2,"International TV Shows, TV Dramas",Netflix
8797,TV Show,Zak Storm,"Michael Johnston, Jessica Gee-George, Christin...",2016,3,Kids' TV,Netflix
8800,TV Show,Zindagi Gulzar Hai,"Sanam Saeed, Fawad Khan, Ayesha Omer, Mehreen ...",2012,1,"International TV Shows, Romantic TV Shows, TV ...",Netflix


In [99]:
TD_1

,type,title,cast,release_year,duration,listed_in,Plataforma
0,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113,"Comedy, Drama",Amazon
1,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110,"Drama, International",Amazon
2,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74,"Action, Drama, Suspense",Amazon
3,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69,Documentary,Amazon
4,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45,"Drama, Fantasy",Amazon
...,...,...,...,...,...,...,...
8802,Movie,Zodiac,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",2007,158,"Cult Movies, Dramas, Thrillers",Netflix
8803,TV Show,Zombie Dumb,None,2018,2,"Kids' TV, Korean TV Shows, TV Comedies",Netflix
8804,Movie,Zombieland,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",2009,88,"Comedies, Horror Movies",Netflix
8805,Movie,Zoom,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",2006,88,"Children & Family Movies, Comedies",Netflix


In [18]:
Total_Data = pd.concat([Amazon, Disney, Hulu, Netflix])

In [21]:
Total_Data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [20]:
Total_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14739 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
dtypes: int64(1), object(11)
memory usage: 2.3+ MB


In [7]:
Amazon.duplicated().sum()

0

In [34]:
Amazon.isnull().sum()

show_id            0
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

In [35]:
Disney.isnull().sum()

show_id           0
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
dtype: int64

In [45]:
Hulu.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [43]:
Disney.duplicated().count()

1450

In [37]:
Hulu.isnull().sum()

show_id            0
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
dtype: int64

In [17]:
Netflix.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64